# Agregar un campo nuevo en models.py
- Sólo es cuestión de agregar un nuevo campo y activar el null de dicho campo, después de eso hacer las migraciones
- python manage.py makemigrations
- python manage.py migrations

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    color = models.TextField()
    seller = models.TextField()
    description = models.TextField()
    short_description = models.TextField(null = True)
    price = models.FloatField()

# Eliminar un campo en models.py
- Basta con sólo eliminarlo de la clase y hacer las migraciones
- python manage.py makemigrations
- python manage.py migrations

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    color = models.TextField()
    seller = models.TextField()
    description = models.TextField()
    price = models.FloatField()

# Borrar migraciones y compresión de migraciones
- python manage.py squashmigrations app numapp
- python manage.py squashmigrations ecommerce 0004
- python manage.py migrate

# Shell Django
- Herramienta util para probar código
- Queryset: Una lista de los objetos almacenados dentro de un modelo

## Obteniendo el qs

In [ ]:
>>> from ecommerce.models import ProductModel
>>> qs = ProductModel.objects.all()
>>> qs
<QuerySet [<ProductModel: ProductModel object (3)>, <ProductModel: ProductModel object (6)>, <ProductModel: ProductModel object (7)>, <ProductModel: ProductModel object (5)>, <ProductModel: ProductModel object (11)>, <ProductModel: ProductModel object (10)>]>

## Obteniendo un objeto en especifico
- Se usa el campo filter y cualquier campo definido en el modelo

In [ ]:
>>> producto = qs.filter(id=5).first()
>>> producto.title
'Cargador'

## Actualizando

In [ ]:
>>> producto.title = 'Cargador Apple'
>>> producto.save()

## Guardando

In [ ]:
>>> pn = ProductModel(title = 'Nuevo Producto', seller = 'Amazon', description = 'Producto Nuevo', price = 300)
>>> pn.save()

## Eliminando

In [ ]:
>>> ProductModel.objects.get(id=12).delete()

# Validación de modelos

## ecommerce/validations.py
- Crear archivo validations.py dentro de la ruta de nuestra app

In [ ]:
from django.core.exceptions import ValidationError

palabrasBloqueadas = [
    "malo",
    "feo",
]

def validacion(valor):
    palabra = valor.lower()
    palabrasUnicas = set(palabra.split())
    bloqueado = set(palabrasBloqueadas)
    invalido = (palabrasUnicas & bloqueado)
    has_error = len(invalido) > 0
    if has_error:
        error = []
        for i in invalido:
            msg = "La palabra: {} no esta permitida".format(i)
            error.append(msg)
        raise ValidationError(error)
    return valor

## ecommerce/models.py
- Importar la validación dentro de nuestros modelos
- Modificar la funcion save() dentro de la clase que realiza el modelo

In [ ]:
from django.db import models
from .validators import validacion
# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    #color = models.TextField()
    seller = models.TextField()
    description = models.TextField()
    price = models.FloatField()

    def save(self,*args,**kwargs):
        validacion(self.title)
        super().save(*args,**kwargs)

# Como agregar opciones a los campos de los modelos

## ecommerce/models.py

In [ ]:
from django.db import models
from .validators import validacion
# Create your models here.

#[(valor en db,valor mostrado)]
status = [
    ('BR','BORRADOR'),
    ('PU','PUBLICADO'),
    ('PR','PRIVADO'),
]
class ProductModel(models.Model):
    title = models.TextField()
    #color = models.TextField()
    seller = models.TextField()
    description = models.TextField()
    price = models.FloatField()
    state = models.CharField(max_length = 2, choices = status, default = 'BR')

    def save(self,*args,**kwargs):
        validacion(self.title)
        super().save(*args,**kwargs)
    
    def is_published(self):
        return self.state == 'PU'

- python manage.py makemigrations
- python manage.py migrate

# Como agregar opciones avanzadas a los campos de los modelos

In [ ]:
from django.db import models
from .validators import validacion
# Create your models here.

class ProductModel(models.Model):
    class ProductState(models.TextChoices):
        PUBLISHED = 'PU','PUBLICO'
        DRAFT = 'BR','BORRADOR'
        PRIVATE = 'PR','PRIVADO'
    title = models.TextField()
    #color = models.TextField()
    seller = models.TextField()
    description = models.TextField()
    price = models.FloatField()
    state = models.CharField(max_length = 2, choices = ProductState.choices, default = ProductState.DRAFT)

    def save(self,*args,**kwargs):
        validacion(self.title)
        super().save(*args,**kwargs)
    
    def is_published(self):
        return self.state == self.ProductState.PUBLISHED

- python manage.py makemigrations
- python manage.py migrate

# Modelo abstracto como base

python manage.py startapp base

## base/models.py

In [ ]:
from django.db import models
from django.utils import timezone
# Create your models here.

class BasePublishModel(models.Model):
    class BasePublishState(models.TextChoices):
        PUBLISHED = 'PU','PUBLICO'
        DRAFT = 'BR','BORRADOR'
        PRIVATE = 'PR','PRIVADO'

    state = models.CharField(max_length = 2, choices = BasePublishState.choices, default = BasePublishState.DRAFT)
    timestamp = models.DateTimeField(auto_now_add = True)
    updated = models.DateTimeField(auto_now_add = True)
    publish_timestamp = models.DateTimeField(auto_now_add = False, auto_now = False , null = True)

    class Meta:
        abstract = True
        ordering = ["-updated","-timestamp"]

    def save(self,*args,**kwargs):
        if self.state_is_published and self.publish_timestamp is None:
            self.publish_timestamp = timezone.now()
        else:
            self.publish_timestamp = None
        super().save(*args,**kwargs)
    @property
    def state_is_published(self):
        return self.state == self.BasePublishState.PUBLISHED
    def is_published(self):
        publish_timestamp = self.publish_timestamp
        return self.state_is_published and publish_timestamp < timezone.now()

## ecommerce/models.py

In [ ]:
from django.db import models
from .validators import validacion
from base.models import BasePublishModel
# Create your models here.

class ProductModel(BasePublishModel):
    title = models.TextField()
    #color = models.TextField()
    seller = models.TextField()
    description = models.TextField()
    price = models.FloatField()

    def save(self,*args,**kwargs):
        validacion(self.title)
        super().save(*args,**kwargs)

## settings.py

In [ ]:
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig",
    "base.apps.BaseConfig",
    "debug_toolbar",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

- python manage.py makemigrations
- python manage.py migratons

# Creación a Granel

In [ ]:
products_data = []
for i in range(1,100):
    new_data = {"title":'Producto {}'.format(i),"price":i*100+99.99}
    products_data.append(new_data)
print(products_data)

In [ ]:
from ecommerce.models import ProductModel
new_objects = []
for product_data in products_data:
    new_objects.append(ProductModel(**product_data))

In [ ]:
ProductModel.objects.bulk_create(new_objects,ignore_conflicts=True)

## SlugField y Señales en modelos

slug: Se usa para los urls

title: escritorio con altura

slug: escritorio-con-altura

## Señales
- pre_save
- post_save
- pre_delete
- post_delete
- pre_init
- post_init
- pre_migrate
- post_migrate

## ecommerce/models.py

In [ ]:
from django.db import models
from django.db.models.signals import pre_save
from .validators import validacion
from base.models import BasePublishModel
from django.utils.text import slugify
#Create your models here.

class ProductModel(BasePublishModel):
    title = models.TextField()
    #color = models.TextField()
    seller = models.TextField()
    description = models.TextField()
    price = models.FloatField()
    slug = models.SlugField(null = True, blank = True, db_index = True)
    def save(self,*args,**kwargs):
        validacion(self.title)
        super().save(*args,**kwargs)
    def get_absolute_url(self):
        return f"/product/{self.slug}"

def pre_save_slug(instance,**kwargs):
    if instance.slug is None or instance.slug == "":
        new_slug = slugify(instance.title)
        myModel = instance.__class__
        qs = myModel.objects.filter(slug__startswith=new_slug).exclude(id=instance.id)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count()}"

pre_save.connect(pre_save_slug,sender=ProductModel)

python manage.py makemigrations

python manage.py migrate

## Fixures para cargar data

python manage.py dumpdata ecommerce --indent 4 --format json > ruta

pyton manage.py loaddata ruta

## Llaves Foraneas

In [ ]:
from django.db import models
from django.conf import settings
from django.db.models.signals import pre_save
from .validators import validacion
from base.models import BasePublishModel
from django.utils.text import slugify
#Create your models here.
User = settings.AUTH_USER_MODEL
class ProductModel(BasePublishModel):
    title = models.TextField()
    #color = models.TextField()
    seller = models.TextField()
    description = models.TextField()
    price = models.FloatField()
    slug = models.SlugField(null = True, blank = True, db_index = True)
    user = models.ForeignKey(User,null = True, on_delete=models.SET_NULL)
    def save(self,*args,**kwargs):
        validacion(self.title)
        super().save(*args,**kwargs)
    def get_absolute_url(self):
        return f"/product/{self.slug}"

def pre_save_slug(instance,**kwargs):
    if instance.slug is None or instance.slug == "":
        new_slug = slugify(instance.title)
        myModel = instance.__class__
        qs = myModel.objects.filter(slug__startswith=new_slug).exclude(id=instance.id)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count()}"

pre_save.connect(pre_save_slug,sender=ProductModel)